In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "cntk"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils


def set_reproducible():
    import tensorflow as tf
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '1960'
    rn.seed(1960)
    np.random.seed(1960)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    import keras.backend.tensorflow_backend as K
    tf.set_random_seed(1960)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

set_reproducible()


Using CNTK backend
/home/antoine/.local/lib/python3.6/site-packages/keras/backend/cntk_backend.py:22: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


# Build a Keras Model

In [2]:
NC = 13

def create_model():
    from keras.models import Sequential
    from keras.layers import Dense, Activation, GRU

    model = Sequential()
    model.add(GRU(1, input_shape=(1, NC), batch_size=1 , 
                  activation='relu', 
                  recurrent_activation = 'hard_sigmoid', 
                  stateful=True))
    model.add(Dense(1, activation=None))
    model.compile(optimizer='adam', loss='mse')
    return model

In [3]:
boston  = datasets.load_boston()
boston.data = (boston.data - boston.data.mean(axis=0)) / boston.data.std(axis=0)
boston.target = (boston.target - boston.target.mean(axis=0)) / boston.target.std(axis=0)

boston.data = boston.data[:,0:NC]
train_X, test_X, train_y, test_y = train_test_split(boston.data, boston.target, train_size=0.8, test_size=0.2, random_state=1960)
train_X = train_X.reshape(train_X.shape[0] , 1, train_X.shape[1])
test_X = test_X.reshape(test_X.shape[0] , 1, test_X.shape[1])



In [4]:
# train_X

In [5]:
#boston.target

In [6]:

from keras.wrappers.scikit_learn import KerasRegressor

clf = KerasRegressor(build_fn=create_model, batch_size=1, epochs=12, verbose=2)

print(train_X.shape , train_y.shape, train_X.dtype , train_y.dtype)
clf.fit(train_X, train_y)

(404, 1, 13) (404,) float64 float64
Epoch 1/12


/usr/local/lib/python3.6/dist-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
/usr/local/lib/python3.6/dist-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input264") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


 - 2s - loss: 0.9163
Epoch 2/12
 - 2s - loss: 0.7540
Epoch 3/12
 - 2s - loss: 0.6947
Epoch 4/12
 - 2s - loss: 0.6641
Epoch 5/12
 - 2s - loss: 0.6396
Epoch 6/12
 - 2s - loss: 0.6258
Epoch 7/12
 - 2s - loss: 0.6286
Epoch 8/12
 - 2s - loss: 0.5992
Epoch 9/12
 - 2s - loss: 0.5841
Epoch 10/12
 - 2s - loss: 0.5770
Epoch 11/12
 - 3s - loss: 0.5781
Epoch 12/12
 - 3s - loss: 0.5634


In [7]:
print(test_X.shape)
preds = clf.predict(test_X)
clf.model.reset_states()
print(preds)


(102, 1, 13)


/usr/local/lib/python3.6/dist-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


[ 0.45985606  0.33687982 -0.21629554  0.02794102  0.24335063  0.09805802
 -0.64036024 -0.21240038 -0.82459545 -1.877132   -1.1097552  -0.72766745
  0.11057565 -0.9000038   0.4871391  -0.5392747   0.4871391   0.4871391
 -1.5523926  -0.2777264  -0.20036495  0.4871391   0.18146977  0.3126344
  0.43195328  0.34240913  0.2662289  -0.83579123 -0.03756952 -1.0595958
  0.2479908  -0.7967409  -1.560562    0.4871391   0.4871391   0.4871391
  0.4871391  -0.8170972  -0.6574148   0.47770151 -0.440414   -0.18311405
  0.01583123 -0.74192834 -0.38740867  0.15573639  0.3224457   0.31580856
 -1.3938138  -0.8883852  -0.45659602 -0.8722843  -1.1685115   0.4871391
 -0.41160762 -0.9313767   0.4871391   0.36005843  0.36296356  0.45265007
  0.32963562  0.43538943  0.450986    0.46490678 -1.4549834   0.3445006
  0.37697324  0.05749318 -0.71920836  0.0906325   0.25545025  0.4699244
  0.47430086  0.48240206  0.00935981 -0.7844994   0.4871391   0.4871391
 -0.65657854  0.4871391   0.28222418  0.37254256  0.4061550

In [8]:
pd.DataFrame(boston.data).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,506.000000,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02,5.060000e+02
mean,0.000000,6.319056e-17,-3.145486e-15,-2.106352e-17,2.752300e-15,-1.150770e-14,-1.137430e-15,7.582867e-16,5.616939e-17,5.616939e-17,-1.022283e-14,8.593916e-15,-5.897786e-16
std,1.000990,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00,1.000990e+00
min,-0.417713,-4.877224e-01,-1.557842e+00,-2.725986e-01,-1.465882e+00,-3.880249e+00,-2.335437e+00,-1.267069e+00,-9.828429e-01,-1.313990e+00,-2.707379e+00,-3.907193e+00,-1.531127e+00
25%,-0.408896,-4.877224e-01,-8.676906e-01,-2.725986e-01,-9.130288e-01,-5.686303e-01,-8.374480e-01,-8.056878e-01,-6.379618e-01,-7.675760e-01,-4.880391e-01,2.050715e-01,-7.994200e-01
50%,-0.388582,-4.877224e-01,-2.110985e-01,-2.725986e-01,-1.442174e-01,-1.084655e-01,3.173816e-01,-2.793234e-01,-5.230014e-01,-4.646726e-01,2.748590e-01,3.811865e-01,-1.812536e-01
75%,0.006248,4.877224e-02,1.015999e+00,-2.725986e-01,5.986790e-01,4.827678e-01,9.067981e-01,6.623709e-01,1.661245e+00,1.530926e+00,8.065758e-01,4.336510e-01,6.030188e-01
max,9.941735,3.804234e+00,2.422565e+00,3.668398e+00,2.732346e+00,3.555044e+00,1.117494e+00,3.960518e+00,1.661245e+00,1.798194e+00,1.638828e+00,4.410519e-01,3.548771e+00


# Generate SQL Code from the Model

In [9]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    cg1.mOptions.mActivateMaterialization = False
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[1]


In [10]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [11]:
lMetaData = {}
lMetaData['features'] = list(boston.feature_names)[0:NC];
lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'boston'

lSQL = test_sql_gen(clf , lMetaData)


cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasRegressor'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.7 None postgresql
KERAS_REG_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f509d5d4048>, 'mFeatureNames': ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], 'mClasses': None, 'mMode': 'Regression'}
{'build_fn': <function create_model at 0x7f509d5c5730>, 'sk_params': {'batch_size': 1, 'epochs': 12, 'verbose': 2}, 'model': <keras.engine.sequential.Sequential object at 0x7f509d5d4208>}
GENERATING_LAYER 0 gru_1 <class 'keras.layers.recurrent.GRU'> {'input_spec': [InputSpec(shape=(1, None, 13), ndim=3)], 'supports_masking': True, 'stateful': True, '_trainable_weights': [], '_non_trainable_weights': [], '_losses': [], '_updates': [(P

In [12]:
print(lSQL)

WITH RECURSIVE keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."CRIM" AS "CRIM", "ADS"."ZN" AS "ZN", "ADS"."INDUS" AS "INDUS", "ADS"."CHAS" AS "CHAS", "ADS"."NOX" AS "NOX", "ADS"."RM" AS "RM", "ADS"."AGE" AS "AGE", "ADS"."DIS" AS "DIS", "ADS"."RAD" AS "RAD", "ADS"."TAX" AS "TAX", "ADS"."PTRATIO" AS "PTRATIO", "ADS"."B" AS "B", "ADS"."LSTAT" AS "LSTAT" 
FROM boston AS "ADS"), 
keras_input_1 AS 
(SELECT keras_input."KEY" AS "KEY", keras_input."CRIM" AS "CRIM", keras_input."ZN" AS "ZN", keras_input."INDUS" AS "INDUS", keras_input."CHAS" AS "CHAS", keras_input."NOX" AS "NOX", keras_input."RM" AS "RM", keras_input."AGE" AS "AGE", keras_input."DIS" AS "DIS", keras_input."RAD" AS "RAD", keras_input."TAX" AS "TAX", keras_input."PTRATIO" AS "PTRATIO", keras_input."B" AS "B", keras_input."LSTAT" AS "LSTAT" 
FROM keras_input), 
keras_input_1_rn AS 
(SELECT row_number() OVER (ORDER BY keras_input_1."KEY" ASC) AS rn, keras_input_1."KEY" AS "KEY", keras_input_1."CRIM" AS "CRIM", keras_input_1."Z

# Execute the SQL Code

In [13]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(boston.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = boston.target
lTable['KEY'] = range(boston.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [14]:
sql_output = pd.read_sql(lSQL , conn);
conn.close()

In [15]:
sql_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,0.487138
112,112,0.220659
125,125,-0.361665
9,9,0.470299
213,213,0.460548
109,109,-0.059821
127,127,-0.913868
244,244,0.487139
406,406,-1.075590
490,490,-1.870520


# Keras Prediction

In [16]:
clf.model.reset_states()
skl_output = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(boston.data.shape[0])), columns=['KEY']);
data = boston.data.reshape(boston.data.shape[0] , 1, boston.data.shape[1])
skl_out = clf.predict(data)
print(skl_out.shape)
skl_output_estimator = pd.DataFrame(skl_out, columns=['Estimator'])
skl_output['KEY'] = skl_output_key['KEY']
skl_output['Estimator'] = skl_output_estimator
skl_output.sample(12, random_state=1960)

/usr/local/lib/python3.6/dist-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


(506,)


,KEY,Estimator
230,230,0.487139
112,112,0.224151
125,125,-0.385935
9,9,0.156321
213,213,0.455469
109,109,-0.035550
127,127,-0.812984
244,244,0.487139
406,406,-0.821306
490,490,-1.940265


# Comparing the SQL and Keras Predictions

In [17]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')
sql_skl_join['Error'] = sql_skl_join.Estimator_sql - sql_skl_join.Estimator_skl

In [18]:
sql_skl_join.head(12)

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
0,0,0.487139,0,0.487139,5.551115e-17
1,1,0.487139,1,0.487139,5.551115e-17
2,2,0.487139,2,0.487139,5.551115e-17
3,3,0.487139,3,0.487139,5.551115e-17
4,4,0.487139,4,0.487139,5.551115e-17
5,5,0.487139,5,0.487139,5.551115e-17
6,6,0.487139,6,0.487139,5.551115e-17
7,7,0.416896,7,0.472469,5.557300e-02
8,8,0.035014,8,0.472469,4.374544e-01
9,9,0.156321,9,0.470299,3.139784e-01


In [19]:
sql_skl_join.describe()

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
count,506.000000,506.000000,506.000000,506.000000,5.060000e+02
mean,252.500000,-0.003612,252.500000,0.010857,1.446880e-02
std,146.213884,0.637648,146.213884,0.623091,5.754076e-02
min,0.000000,-2.331509,0.000000,-2.146417,-2.542834e-01
25%,126.250000,-0.471313,126.250000,-0.475320,-2.198941e-03
50%,252.500000,0.395747,252.500000,0.434268,-3.941292e-15
75%,378.750000,0.486967,378.750000,0.485663,1.192279e-02
max,505.000000,0.487139,505.000000,0.487139,4.374544e-01
